In [1]:
import numpy as np
import jax
import jax.numpy as jnp
import flax.linen as nn
import matplotlib.pyplot as plt
import os
from tifffile import imread, imsave
import torch
import torch.utils.data as dt

In [2]:
os.chdir('/mnt/c/Users/ankit/Desktop/Msc AIML/msc-project/GAP-JAX')
# os.chdir(r'C:\Users\ankit\Desktop\Msc AIML\msc-project\GAP-JAX')
from gap_jax.BinomDataset_JAX import BinomDataset
from  gap_jax.GAP_JAX_Trainer import Trainer
from gap_jax.GAP_UNET_ResBlock_JAX import UN

In [3]:
data = imread('/mnt/c/Users/ankit/Desktop/Msc AIML/Data/convallaria/trainingDataGT.tif')
# data = imread(r'C:\Users\ankit\Desktop\Msc AIML\Data\convallaria\trainingDataGT.tif')

In [4]:
print(data.shape)

(1536, 512, 512)


In [5]:
def psnrToString(inp):
    if inp < 0:
        return 'm'+str(-inp)
    else:
        return str(inp)

minpsnr = -40
maxpsnr = -5

name = psnrToString(minpsnr)+"to"+psnrToString(maxpsnr)+"-256x256-jax-v1"
# CHECKPOINT_PATH = '/mnt/c/Users/ankit/Desktop/Msc AIML/Data/checkpoints/models/jax/wsl/'
# CHECKPOINT_PATH  = "C:\\Users\\ankit\\Desktop\\Msc AIML\\Data\\checkpoints\\models\\jax\\windows\\"
CHECKPOINT_PATH = '/home/ankith/temp/'
CHECKPOINT_PATH, name

('/home/ankith/temp/', 'm40tom5-256x256-jax-v1')

In [7]:
maxepochs = 25 # train longer for better results

# Create pytorch dataset
dataset = BinomDataset(data = data[:round(data.shape[0]*0.9)], windowSize = 256, minPSNR = minpsnr, maxPSNR = maxpsnr)
val_dataset = BinomDataset(data = data[round(data.shape[0]*0.9):], windowSize = 256, minPSNR = minpsnr, maxPSNR = maxpsnr)

# Create pytorch dataloader for training and validation sets
train_loader = dt.DataLoader(dataset, batch_size=4, shuffle=True, drop_last=True, pin_memory=False, num_workers=4, collate_fn= lambda batch: np.array(batch)) ## Changing the batch size from 32 to 16 to fit inside the gpu
val_loader = dt.DataLoader(val_dataset, batch_size=4, shuffle=False, drop_last=True,  pin_memory=False, num_workers=4, collate_fn= lambda batch: np.array(batch))

# img =  next(iter(val_loader))

# print(img.shape)

In [8]:
model = UN(channels = 1, levels=10, depth=3, start_filts=28, up_mode = 'transpose')

In [9]:
trainer = Trainer(root_dir= os.path.join(CHECKPOINT_PATH, name),
                  gradient_clip_val=0.5,
                  epochs= maxepochs,
                  dataloader= train_loader,
                  model = model)

GPU detected with 1 Device(s).


In [10]:
trainer.train_model(train_loader= train_loader,
                    val_loader= val_loader)

  0%|          | 0/25 [00:00<?, ?it/s]

/home/ankith/anaconda3/envs/rsenv/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Training:   0%|          | 0/345 [00:00<?, ?it/s]

2024-07-20 12:26:55.306031: W external/xla/xla/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 14.63GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
/home/ankith/anaconda3/envs/rsenv/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/ankith/anaconda3/envs/rsenv/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/ankith/anaconda3/envs/rsenv/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded co

Training:   0%|          | 0/345 [00:00<?, ?it/s]

...

Training:   0%|          | 0/345 [00:00<?, ?it/s]

Training:   0%|          | 0/345 [00:00<?, ?it/s]

Training:   0%|          | 0/345 [00:00<?, ?it/s]

Training:   0%|          | 0/345 [03:20<?, ?it/s]

Met early stopping criteria, breaking at epoch 21


In [12]:
trainer.state.params

{'params': {'conv_final': {'bias': Array([7.229851e-05], dtype=float32),
   'kernel': Array([[[-0.3401088 ],
           [-0.09934841],
           [-0.2879025 ],
           [ 0.16942768],
           [-0.13767822],
           [ 0.30031532],
           [ 0.02246707],
           [-0.15176843],
           [ 0.0971997 ],
           [-0.11484206],
           [ 0.3896479 ],
           [ 0.11628883],
           [ 0.11111325],
           [-0.05902993],
           [ 0.12670632],
           [-0.25139603],
           [-0.34526727],
           [ 0.1429715 ],
           [-0.10612093],
           [ 0.25510514],
           [-0.18146142],
           [-0.32377964],
           [ 0.15408282],
           [ 0.0560473 ],
           [ 0.15207882],
           [ 0.41178092],
           [-0.35273266],
           [-0.10898618]]], dtype=float32)},
  'down_convs_0': {'conv1': {'bias': Array([-0.00018352,  0.00178131, -0.00055144, -0.00165127,  0.0010285 ,
            0.00181668, -0.00046491,  0.0003527 , -0.00012807

In [8]:
# img = jnp.ones((1, 256, 256, 1))
img = next(iter(train_loader))[:,:,:,1:]
img.shape

(1, 256, 256, 1)

In [9]:
params = model.init(jax.random.key(68), img)

In [10]:
params

{'params': {'down_convs_0': {'conv1': {'kernel': Array([[[[ 0.04835897, -0.01130721,  0.13503538, ...,  0.06476042,
              -0.00980399,  0.13382831],
             [-0.11486074, -0.06955706,  0.06231094, ...,  0.00211302,
              -0.06623932, -0.12104253],
             [-0.08883706, -0.04109567,  0.06634794, ..., -0.11131057,
              -0.05823373, -0.10369347],
             ...,
             [ 0.1004604 , -0.05761833,  0.10426241, ..., -0.07090598,
              -0.05718922, -0.0832227 ],
             [ 0.15443651, -0.02431712, -0.054013  , ...,  0.00984212,
              -0.04689721, -0.06521375],
             [ 0.07888078,  0.0981305 , -0.10198117, ...,  0.06014301,
              -0.15116563, -0.07633726]],
    
            [[-0.13767436, -0.13606043,  0.06190977, ...,  0.00630517,
               0.0546028 ,  0.08283786],
             [ 0.05103631, -0.03247613, -0.05663961, ..., -0.02367956,
              -0.08064031, -0.07050055],
             [ 0.13343303,  0.05339

In [11]:
out1 = model.apply(params, img)

In [12]:
np.unique(out1 == out1)

array([ True])

In [13]:

CHECKPOINT_PATH, name

('/mnt/c/Users/ankit/Desktop/Msc AIML/Data/checkpoints/models/jax/wsl/',
 'm40tom5-256x256-jax-v1')

In [13]:
from flax.training import checkpoints
# CHECKPOINT_PATH = '/mnt/c/Users/ankit/Desktop/Msc AIML/Data/checkpoints/models/jax/del/'
CHECKPOINT_PATH = "/home/ankith/temp/test/"
if not os.path.exists(CHECKPOINT_PATH):
    os.makedirs(CHECKPOINT_PATH)
checkpoints.save_checkpoint(ckpt_dir=CHECKPOINT_PATH,
                                    target=trainer.state.params,
                                    step = 0,
                                    overwrite=True)

'/home/ankith/temp/test/checkpoint_0'

In [18]:
type(params)

dict